# <center>Multivariate Linear Regression on House Pricing Data</center>


### <center>Source: Kaggle - House Prices: Advanced Regression Techniques
<center> https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LinearRegression


%load_ext autoreload
%autoreload 2

## Data Engineering

In [2]:
traindf = pd.read_csv('train.csv')

In [3]:
drop_list = ['Exterior2nd', 'YearRemodAdd', 'LotShape', 'MSSubClass', 'TotalBsmtSF', 'Alley', 'GarageYrBlt', 'PoolQC', 'FireplaceQu', 'MiscFeature', 'MiscFeature']

#### Moving Sale Price over to targetdf

In [4]:
targetdf = traindf[['Id', 'SalePrice']]
traindf.drop('SalePrice', axis=1, inplace=True)

#### Removing problematic columns

In [5]:
traindf = traindf.drop(drop_list, axis=1)

#### Seperating the catagorical and numerical data

In [6]:
traindf_cat = traindf[['Id', 'MSZoning']].copy()
traindf.drop('MSZoning', axis=1, inplace=True)

In [7]:
lotfront = traindf.LotFrontage.mean()

Replacing NA values with the mean value for the column

In [8]:
traindf.LotFrontage.fillna(value=lotfront, inplace=True)

In [9]:
traindf_cat['Street'] = traindf.Street
traindf.drop('Street', axis=1, inplace=True)

In [10]:
traindf_cat['LandContour'] = traindf.LandContour
traindf.drop('LandContour', axis=1, inplace=True)

In [11]:
traindf_cat['Utilities'] = traindf.Utilities
traindf.drop('Utilities', axis=1, inplace=True)

In [12]:
traindf_cat['LotConfig'] = traindf.LotConfig
traindf.drop('LotConfig', axis=1, inplace=True)

In [13]:
traindf_cat['LandSlope'] = traindf.LandSlope
traindf.drop('LandSlope', axis=1, inplace=True)

In [14]:
traindf_cat['Neighborhood'] = traindf.Neighborhood
traindf.drop('Neighborhood', axis=1, inplace=True)

In [15]:
traindf_cat['Condition1'] = traindf.Condition1
traindf.drop('Condition1', axis=1, inplace=True)

traindf_cat['Condition2'] = traindf.Condition2
traindf.drop('Condition2', axis=1, inplace=True)

In [16]:
traindf_cat['BldgType'] = traindf.BldgType
traindf.drop('BldgType', axis=1, inplace=True)

In [17]:
traindf_cat['HouseStyle'] = traindf.HouseStyle
traindf.drop('HouseStyle', axis=1, inplace=True)

In [18]:
traindf_cat['RoofStyle'] = traindf.RoofStyle
traindf.drop('RoofStyle', axis=1, inplace=True)

In [19]:
traindf_cat['RoofMatl'] = traindf.RoofMatl
traindf.drop('RoofMatl', axis=1, inplace=True)

In [20]:
traindf_cat['Exterior1st'] = traindf.Exterior1st
traindf.drop('Exterior1st', axis=1, inplace=True)

#traindf_cat['Exterior2nd'] = traindf.Exterior2nd
#traindf.drop('Exterior2nd', axis=1, inplace=True)

In [21]:
traindf.MasVnrType.fillna(value='None', inplace=True)
traindf_cat['MasVnrType'] = traindf.MasVnrType
traindf.drop('MasVnrType', axis=1, inplace=True)

In [22]:
traindf.ExterQual = traindf.ExterQual.replace({'Ex': 10., 'Gd':7., 'TA':4., 'Fa':2, 'Po':1.}, value=None)

In [23]:
traindf.ExterCond = traindf.ExterCond.replace({'Ex': 10., 'Gd':7., 'TA':4., 'Fa':2, 'Po':1.}, value=None)

In [24]:
traindf_cat['Foundation'] = traindf.Foundation
traindf.drop('Foundation', axis=1, inplace=True)

In [25]:
traindf.MasVnrArea.fillna(value=0.00, inplace=True) 

In [26]:
traindf.BsmtCond = traindf.BsmtCond.replace({'Ex': 10, 'Gd':7, 'TA':4, 'Fa':2, 'Po':1}, value=None)
traindf.BsmtCond.fillna(value=0, inplace=True)

In [27]:
traindf.BsmtFinType1 = traindf.BsmtFinType1.replace({'GLQ': 10, 'ALQ':7, 'BLQ':5, 'Rec':4, 'LwQ':2, 'Unf':1}, value=None)
traindf.BsmtFinType1.fillna(value=0, inplace=True)

In [28]:
traindf.BsmtExposure = traindf.BsmtExposure.replace({'Gd':10, 'Av': 7, 'Mn':2, 'No':1}, value=None)
traindf.BsmtExposure.fillna(value=0, inplace=True)

In [29]:
traindf.BsmtQual = traindf.BsmtQual.replace({'Ex':10, 'Gd':7, 'TA': 4, 'Fa':2, 'Po':1}, value=None)
traindf.BsmtQual.fillna(value=0, inplace=True)

In [30]:
traindf.BsmtFinType2 = traindf.BsmtFinType2.replace({'GLQ': 10, 'ALQ':7, 'BLQ':5, 'Rec':5, 'LwQ':2, 'Unf':1}, value=None)
traindf.BsmtFinType2.fillna(value=0, inplace=True)

In [31]:
traindf_cat['Heating'] = traindf.Heating
traindf.drop('Heating', axis=1, inplace=True)

In [32]:
traindf.HeatingQC = traindf.HeatingQC.replace({'Ex': 10, 'Gd':7, 'TA':4, 'Fa':2, 'Po':1}, value=None)
traindf.HeatingQC.fillna(value=0, inplace=True)

In [33]:
traindf_cat['CentralAir'] = traindf.CentralAir
traindf.drop('CentralAir', axis=1, inplace=True)

In [34]:
traindf_cat['Electrical'] = traindf.Electrical
traindf.drop('Electrical', axis=1, inplace=True)
traindf_cat.Electrical.fillna(value='SBrkr', inplace=True)

In [35]:
traindf.KitchenQual = traindf.KitchenQual.replace({'Ex': 10, 'Gd':7, 'TA':5, 'Fa':3, 'Po':1}, value=None)
traindf.KitchenQual.fillna(value=1, inplace=True)

In [36]:
traindf_cat['Functional'] = traindf.Functional
traindf.drop('Functional', axis=1, inplace=True)

In [37]:
traindf_cat['GarageType'] = traindf.GarageType
traindf.drop('GarageType', axis=1, inplace=True)
traindf_cat.GarageType.fillna(value='NoGarage', inplace=True)

In [38]:
traindf.GarageFinish = traindf.GarageFinish.replace({'Fin': 7, 'RFn':4, 'Unf':1}, value=None)
traindf.GarageFinish.fillna(value=0., inplace=True)

In [39]:
traindf.GarageQual = traindf.GarageQual.replace({'Ex': 10, 'Gd':7, 'TA':4, 'Fa':2, 'Po':1}, value=None)
traindf.GarageQual.fillna(value=0, inplace=True)

In [40]:
traindf.GarageCond = traindf.GarageCond.replace({'Ex': 10, 'Gd':7, 'TA':4, 'Fa':2, 'Po':1}, value=None)
traindf.GarageCond.fillna(value=0, inplace=True)

In [41]:
traindf_cat['PavedDrive'] = traindf.PavedDrive
traindf.drop('PavedDrive', axis=1, inplace=True)

In [42]:
traindf_cat['Fence'] = traindf.Fence
traindf.drop('Fence', axis=1, inplace=True)
traindf_cat.fillna(value='NoFence', inplace=True)

In [43]:
traindf_cat['MoSold'] = traindf.MoSold
traindf.drop('MoSold', axis=1, inplace=True)

In [44]:
traindf_cat['SaleType'] = traindf.SaleType
traindf.drop('SaleType', axis=1, inplace=True)

In [45]:
traindf_cat['SaleCondition'] = traindf.SaleCondition
traindf.drop('SaleCondition', axis=1, inplace=True)

In [46]:
traindf['ExterQual'] = traindf.ExterQual.astype(int)
traindf['ExterCond'] = traindf.ExterCond.astype(int)

In [47]:
## Add below to catagorical -- and add to .py module

In [48]:
traindf_cat['BsmtFullBath'] = traindf.BsmtFullBath
traindf.drop('BsmtFullBath', axis=1, inplace=True)

In [49]:
traindf_cat['BsmtHalfBath'] = traindf.BsmtHalfBath
traindf.drop('BsmtHalfBath', axis=1, inplace=True)

In [50]:
traindf_cat['FullBath'] = traindf.FullBath
traindf.drop('FullBath', axis=1, inplace=True)

In [51]:
traindf_cat['HalfBath'] = traindf.HalfBath
traindf.drop('HalfBath', axis=1, inplace=True)

In [52]:
traindf_cat['BedroomAbvGr'] = traindf.BedroomAbvGr
traindf.drop('BedroomAbvGr', axis=1, inplace=True)

In [53]:
traindf_cat['KitchenAbvGr'] = traindf.KitchenAbvGr
traindf.drop('KitchenAbvGr', axis=1, inplace=True)

In [54]:
traindf_cat['TotRmsAbvGrd'] = traindf.TotRmsAbvGrd
traindf.drop('TotRmsAbvGrd', axis=1, inplace=True)

In [55]:
traindf_cat['Fireplaces'] = traindf.Fireplaces
traindf.drop('Fireplaces', axis=1, inplace=True)

In [56]:
traindf_cat['GarageCars'] = traindf.GarageCars
traindf.drop('GarageCars', axis=1, inplace=True)

In [57]:
traindf['ThreeSsnPorch'] = traindf['3SsnPorch']
traindf.drop('3SsnPorch', axis=1, inplace=True)

In [58]:
traindf['FirstFlrSF'] = traindf['1stFlrSF']
traindf.drop('1stFlrSF', axis=1, inplace=True)

In [59]:
traindf['SecondFlrSF'] = traindf['2ndFlrSF']
traindf.drop('2ndFlrSF', axis=1, inplace=True)

## Feature Scaling

In [60]:
traindf_columns = traindf.columns

In [61]:
from sklearn.preprocessing import StandardScaler
from scipy import stats

ss = StandardScaler()

traindfss = pd.DataFrame(ss.fit_transform(traindf))
traindfss.set_index(traindf.Id, inplace=True)
traindfss.columns = traindf_columns 
# Remove outliers, i.e. data points outside of 2.5 standard deviations of the mean
traindfss = traindfss[(np.abs(stats.zscore(traindfss)) < 2.5).all(axis=1)]
#X_train_numeric.head()

In [62]:
traindfss.drop('Id', axis=1, inplace=True)

In [63]:
traindfss.columns

Index(['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'HeatingQC', 'LowQualFinSF', 'GrLivArea',
       'KitchenQual', 'GarageFinish', 'GarageArea', 'GarageQual', 'GarageCond',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'YrSold', 'ThreeSsnPorch', 'FirstFlrSF', 'SecondFlrSF'],
      dtype='object')

## Getting dummies

In [64]:
traindf_cat.Exterior1st.unique()

array(['VinylSd', 'MetalSd', 'Wd Sdng', 'HdBoard', 'BrkFace', 'WdShing',
       'CemntBd', 'Plywood', 'AsbShng', 'Stucco', 'BrkComm', 'AsphShn',
       'Stone', 'ImStucc', 'CBlock'], dtype=object)

In [65]:
traindf_cat.HouseStyle= traindf_cat.HouseStyle.replace({'1.5Fin': '1.5_Fin', '1.5Unf': '1_5Unf', '2.5Unf':'2_5Unf', '2.5Fin':'2_5Fin'}, value=None)

In [66]:
traindf_cat.Exterior1st= traindf_cat.Exterior1st.replace({'Wd Sdng': 'Wd_Sdng'}, value=None)

In [67]:
traindf_cat.RoofMatl= traindf_cat.RoofMatl.replace({'Tar&Grv': 'TarGrv'}, value=None)

In [68]:
cat_list = list(traindf_cat.columns)[1:]

In [69]:
traindf_cat_bined= pd.get_dummies(traindf_cat, columns=cat_list, drop_first=True)

In [70]:
columnfixlist = [traindf_cat_bined.columns[i] for i in range(len(traindf_cat_bined.columns))]
columnfixlist

['Id',
 'MSZoning_FV',
 'MSZoning_RH',
 'MSZoning_RL',
 'MSZoning_RM',
 'Street_Pave',
 'LandContour_HLS',
 'LandContour_Low',
 'LandContour_Lvl',
 'Utilities_NoSeWa',
 'LotConfig_CulDSac',
 'LotConfig_FR2',
 'LotConfig_FR3',
 'LotConfig_Inside',
 'LandSlope_Mod',
 'LandSlope_Sev',
 'Neighborhood_Blueste',
 'Neighborhood_BrDale',
 'Neighborhood_BrkSide',
 'Neighborhood_ClearCr',
 'Neighborhood_CollgCr',
 'Neighborhood_Crawfor',
 'Neighborhood_Edwards',
 'Neighborhood_Gilbert',
 'Neighborhood_IDOTRR',
 'Neighborhood_MeadowV',
 'Neighborhood_Mitchel',
 'Neighborhood_NAmes',
 'Neighborhood_NPkVill',
 'Neighborhood_NWAmes',
 'Neighborhood_NoRidge',
 'Neighborhood_NridgHt',
 'Neighborhood_OldTown',
 'Neighborhood_SWISU',
 'Neighborhood_Sawyer',
 'Neighborhood_SawyerW',
 'Neighborhood_Somerst',
 'Neighborhood_StoneBr',
 'Neighborhood_Timber',
 'Neighborhood_Veenker',
 'Condition1_Feedr',
 'Condition1_Norm',
 'Condition1_PosA',
 'Condition1_PosN',
 'Condition1_RRAe',
 'Condition1_RRAn',
 'Con

In [71]:
train_combined = pd.merge(traindf_cat_bined, traindfss, on='Id', left_index=True, right_index=True)

In [72]:
targetdf = targetdf.loc[train_combined.index]

In [73]:
train_combined.drop('Id', axis=1, inplace=True)

In [74]:
targetdf.set_index('Id', inplace=True)

In [75]:
targetdf.head()

,SalePrice
Id,
1,208500
2,181500
3,223500
4,140000
5,250000


## Testing for colinearity amonst columns

### Testing for intercorrelation

In [76]:
col_names = list(train_combined.columns)

In [77]:
colinear = []
for ax1 in col_names:
    for ax2 in col_names:
        if ax1 == ax2:
            break
        else:
            if train_combined[f'{ax1}'].corr(train_combined[f'{ax2}']) >= .75:
                colinear.append((ax1,ax2))

In [78]:
colinear

[('Neighborhood_Somerst', 'MSZoning_FV'),
 ('SaleCondition_Partial', 'SaleType_New'),
 ('TotRmsAbvGrd_14', 'BedroomAbvGr_8'),
 ('BsmtQual', 'YearBuilt'),
 ('BsmtFinSF2', 'BsmtFinType2')]

### Testing for correlation with Target - SalePrice

In [100]:
corr_saleprice = {}
for i in col_names:
    corr_saleprice[i] = train_combined[f'{i}'].corr(targetdf['SalePrice'])

In [130]:
corr_table = pd.DataFrame( corr_saleprice.values(), index = corr_saleprice.keys())
corr_table.describe()

,0
count,211.000000
mean,-0.001347
std,0.033583
min,-0.071093
25%,-0.024287
50%,-0.005155
75%,0.019415
max,0.114786


### Optimizing parameters - How can I test all the combinations of assigned values for each column to yield the best parameters for each column

In [80]:
#traindf.BsmtCond = traindf.BsmtCond.replace({'Ex': 10, 'Gd':7, 'TA':4, 'Fa':2, 'Po':1}, value=None)
##traindf.BsmtCond = traindf.BsmtCond.replace({10: a, 7:b, 4:c, 2:d, 1:e}, value=None)

### Optimizing the degree of variables

In [90]:
#for y in col_names:
 #   plt.figure()
  #  plt.scatter(x=targetdf['SalePrice'], y=train_combined[f'{y}'])
   # plt.xlabel('SalePrice')
    #plt.ylabel(f'{y}')

### Testing the linerar regression

In [82]:
lr = LinearRegression()
lr.fit(train_combined, targetdf)
y_hat_train = lr.predict(train_combined)
print(lr.coef_, train_combined.columns)
#print(train_combined.columns)

[[ 3.03315933e+04  5.52940381e+04  5.10232380e+04  4.37758300e+04
   1.69562232e+04  5.10254496e+03  7.73188132e+03 -4.02041869e+03
   3.36248473e+16  1.44682500e+04 -1.24206250e+04 -2.71079141e+04
   2.13000000e+02  7.00112500e+03  7.71562500e+03  1.01752500e+04
   1.78100000e+04 -1.17228750e+04 -1.87755625e+04 -7.83400000e+03
  -6.04632812e+02 -2.78792500e+04 -2.42642500e+04 -4.56900000e+03
  -2.17918750e+04 -1.13787109e+04 -2.25595000e+04  2.70437500e+02
  -2.21443125e+04  3.54690000e+04  4.48695000e+04 -1.06425000e+04
  -1.45097500e+04 -2.01636250e+04  4.40062500e+03  3.44411875e+04
   5.67936250e+04 -4.62075000e+03 -5.71657227e+03  2.39375000e+02
   1.59220000e+04  2.27885664e+04  2.63968750e+04 -7.92152246e+03
   2.21890781e+04  1.43186875e+04  2.79514922e+04 -4.19263281e+03
  -7.11637500e+03  5.51620312e+04 -2.03906836e+05  2.38944101e+15
  -4.49773567e+14 -3.14140801e+04 -2.40000000e+03  5.14016406e+03
  -3.90792500e+04 -3.01104375e+04  7.32000000e+02 -6.42278906e+03
  -1.11180

In [83]:
from sklearn.metrics import r2_score, mean_squared_error

print(f"r^2: {r2_score(targetdf, y_hat_train)}")
print(f"rmse: {np.sqrt(mean_squared_error(targetdf, y_hat_train))}")

r^2: 0.8985402777292189
rmse: 25772.688532876182


In [84]:
### Previous R^2 values ###


### Added Rooms, Car garage, fireplaces and some other discrete values to the catagorical side of the table... increased features
### by approx 30 --- need to check R^2 adj 
# r^2: 0.898540213574302
# rmse: 25772.696681156067

### Replaced lot frontage with mean value instead of '0'
# r^2: 0.8175929129184736
# rmse: 34972.641412123485


### Initial R^2 to beat
# r^2: 0.6589675323
# rmse: 36779.91592177666

In [85]:
import statsmodels.formula.api as sm
train_combined_string = ''
for i in [train_combined.columns[i] for i in range(len(train_combined.columns))]:
    train_combined_string += i + ' + ' 
train_combined_string = train_combined_string[0:-3]

sm_formula = ("SalePrice ~ " + train_combined_string ) 
ols_data = pd.merge(targetdf, train_combined, left_index=True, right_index=True)

results = sm.ols(formula = sm_formula, data = ols_data).fit()

In [86]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.245
Model:                            OLS   Adj. R-squared:                 -0.034
Method:                 Least Squares   F-statistic:                    0.8783
Date:                Mon, 06 Jan 2020   Prob (F-statistic):              0.863
Time:                        16:19:07   Log-Likelihood:                -9586.2
No. Observations:                 762   AIC:                         1.958e+04
Df Residuals:                     556   BIC:                         2.054e+04
Df Model:                         205                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept             -2.207e+04   1.18e+05     -0.187      0.852   -2.54e+05     2.1e+05
MSZoning_FV            9.728e+04   6.85e+04      1.420      0.156   -3.73e+04    2.32e+05
MSZoning_RH             9.53e+04   6.68e+04      1.426      0.154    -3.6e+04    2.27e+05
MSZoning_RL            7.297e+04    5.3e+04      1.377      0.169   -3.11e+04    1.77e+05
MSZoning_RM            6.568e+04   4.89e+04      1.344      0.180   -3.03e+04    1.62e+05
Street_Pave           -6.512e+04   7.17e+04     -0.908      0.364   -2.06e+05    7.58e+04
LandContour_HLS       -5.009e+04   2.79e+04     -1.796      0.073   -1.05e+05    4690.521
LandContour_Low       -6.013e+04   3.77e+04     -1.596      0.111   -1.34e+05    1.39e+04
LandContour_Lvl       -1.769e+04   2.02e+04     -0.874      0.383   -5.75e+04    2.21e+04
Utilities_NoSeWa      -1.635e-09   1.61e-09     -1.018      0.309   -4.79e-09    1.52e-09
LotConfig_CulDSac     -8677.8203   1.64e+04     -0.530      0.596   -4.08e+04    2.35e+04
LotConfig_FR2         -3.616e+04   2.13e+04     -1.700      0.090   -7.79e+04    5612.018
LotConfig_FR3          9.171e+04   7.86e+04      1.166      0.244   -6.28e+04    2.46e+05
LotConfig_Inside      -3608.9423   9013.568     -0.400      0.689   -2.13e+04    1.41e+04
LandSlope_Mod           3.66e+04   2.51e+04      1.458      0.145   -1.27e+04    8.59e+04
LandSlope_Sev          1.224e+04   4.58e+04      0.267      0.789   -7.77e+04    1.02e+05
Neighborhood_Blueste   3.003e+04   1.06e+05      0.285      0.776   -1.77e+05    2.37e+05
Neighborhood_BrDale    4.254e+04    5.7e+04      0.746      0.456   -6.95e+04    1.55e+05
Neighborhood_BrkSide    696.5968   4.93e+04      0.014      0.989   -9.62e+04    9.76e+04
Neighborhood_ClearCr  -3.754e+04   4.98e+04     -0.753      0.452   -1.35e+05    6.04e+04
Neighborhood_CollgCr  -1.975e+04   4.01e+04     -0.493      0.623   -9.85e+04     5.9e+04
Neighborhood_Crawfor  -5.078e+04   4.64e+04     -1.095      0.274   -1.42e+05    4.03e+04
Neighborhood_Edwards  -2.185e+04   4.27e+04     -0.512      0.609   -1.06e+05     6.2e+04
Neighborhood_Gilbert  -1.288e+04   4.26e+04     -0.302      0.763   -9.66e+04    7.08e+04
Neighborhood_IDOTRR    -1.14e+04   5.57e+04     -0.205      0.838   -1.21e+05     9.8e+04
Neighborhood_MeadowV  -9764.1687   5.99e+04     -0.163      0.871   -1.27e+05    1.08e+05
Neighborhood_Mitchel   2.232e+04   4.47e+04      0.500      0.618   -6.54e+04     1.1e+05
Neighborhood_NAmes     -2.44e+04   4.29e+04     -0.568      0.570   -1.09e+05    5.99e+04
Neighborhood_NPkVill  -7552.8421   5.64e+04     -0.134      0.893   -1.18e+05    1.03e+05
Neighborhood_NWAmes   -7131.2818   4.41e+04     -0.162      0.872   -9.38e+04    7.95e+04
Neighborhood_NoRidge  -4.393e+04   4.44e+04     -0.990      0.322   -1.31e+05    4.32e+04
Neighborhood_NridgHt   

In [87]:
### Ridge and Lasso 

In [91]:
#### Lasso

In [145]:
#### Elastic Net

In [176]:
from sklearn.linear_model import Lasso

Lassoreg = Lasso(alpha=2, max_iter = 1000000000 ,normalize = False)
Lassoreg.fit(train_combined, targetdf)
L_target_prediction = ENreg.predict(train_combined)  #### TOL parameter
lassCoef = Lassoreg.coef_

In [177]:
targetdf['SalePrice'] - L_target_prediction

Id
1     -10370.218882
2      14887.355844
3       4554.844114
4     -52273.129136
5     -49888.230696
           ...     
759    -8400.890349
760   -10671.223198
761     7177.861957
762      695.256774
763    19850.377224
Name: SalePrice, Length: 763, dtype: float64

In [183]:
lassCoef = Lassoreg.coef_

array([ 2.71433736e+04,  5.25355817e+04,  4.84227013e+04,  4.13757275e+04,
        1.58142779e+04,  4.38262801e+03,  4.95343658e+03, -4.20287258e+03,
        0.00000000e+00,  1.45222428e+04, -1.21445151e+04, -2.48513242e+04,
        1.22314521e+02,  6.42129423e+03,  8.19045711e+03,  8.47965249e+03,
        1.82677743e+04, -1.11108847e+04, -1.57077703e+04, -6.10027683e+03,
        1.09400409e+03, -2.58750624e+04, -2.25776319e+04, -2.48931248e+03,
       -2.02080644e+04, -1.00467252e+04, -2.09716325e+04,  5.05490126e+02,
       -2.03452011e+04,  3.69139770e+04,  4.64395232e+04, -9.81810035e+03,
       -1.24780309e+04, -1.84319769e+04,  6.04723854e+03,  3.63482682e+04,
        5.87371409e+04, -2.70597785e+03, -3.16517911e+03, -1.45793663e+03,
        1.44528837e+04,  1.97028396e+04,  2.39861691e+04, -9.01940451e+03,
        2.09219766e+04,  1.19512465e+04,  2.43133528e+04,  0.00000000e+00,
       -2.30991793e+03,  5.53750411e+04, -1.95981805e+05,  0.00000000e+00,
        0.00000000e+00, -

In [179]:
from sklearn.linear_model import ElasticNet

ENreg = ElasticNet(alpha=.1,l1_ratio= .5, normalize = False)
ENreg.fit(train_combined, targetdf)
target_prediction = ENreg.predict(train_combined)

In [171]:
targetdf['SalePrice'] - target_prediction

Id
1     -10370.218882
2      14887.355844
3       4554.844114
4     -52273.129136
5     -49888.230696
           ...     
759    -8400.890349
760   -10671.223198
761     7177.861957
762      695.256774
763    19850.377224
Name: SalePrice, Length: 763, dtype: float64

In [180]:
prediction

AttributeError: 'numpy.ndarray' object has no attribute 'plt'